In [ ]:
%matplotlib notebook
from IPython.display import Audio as Play
from IPython.display import display
from scipy.io import wavfile
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal.windows import hann
from scipy import signal

In [ ]:
# read file
path = "beat.wav"
samplerate, data = wavfile.read(path)
data = data[:, 0]/2**16  # scale between -1 and 1

In [ ]:
# plot and play signal
def show_signal(x, samplerate):
    plt.figure()
    plt.plot(np.arange(len(x))/samplerate, x)
    plt.xlabel("time(s)")
    plt.grid(True)
    return Play(x, rate=samplerate)

In [ ]:
show_signal(data, samplerate)

In [ ]:
# we use pandas to have fast frame based processing with little code :)
win_size = 2048
overlap = 1024
idx_frame = np.arange(0, len(data)-win_size, overlap)
df_frames = pd.DataFrame(data=idx_frame, columns=["frame_start"])
df_frames["frames"] = df_frames["frame_start"].apply(lambda x: data[x:(x+win_size)])
df_frames = df_frames.set_index("frame_start", drop=True)
df_frames

In [ ]:
# extract rms per frame
window = hann(win_size)
df_frames["rms"] = df_frames["frames"].apply(lambda x: np.sqrt(np.mean((x*window)**2)))
plt.figure()
plt.plot(data, label="signal")
plt.plot(df_frames["rms"], label="rms")
plt.legend()
df_frames

In [ ]:
# let's attach the rms to a lowpass filter cutoff frequency :)
# first convert rms to a cutoff value
cutoff = (df_frames["rms"]-df_frames["rms"].min()) / (df_frames["rms"].max() - df_frames["rms"].min())
cutoff_min = 250
cutoff_max = 20000
df_frames["cutoff"] = cutoff*(cutoff_max - cutoff_min)
df_frames["cutoff"] = df_frames["cutoff"].clip(cutoff_min, cutoff_max)
df_frames.plot(subplots=True, grid=True)
df_frames

In [ ]:
# now filter each frame
def filter_signal(data):
    frame = data["frames"]
    cutoff = data["cutoff"]
    w = cutoff / (samplerate / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    return signal.lfilter(b, a, frame)

df_frames["frames_filtered"] = df_frames[["frames", "cutoff"]].apply(filter_signal, axis=1)
df_frames

In [ ]:
# overlapp add signals windowed
y = np.zeros(len(data))
def overlap_add(x):
    idx_start = x.name
    idx_end = idx_start + win_size
    idx_end = np.min([idx_end, len(y)])
    y[idx_start:idx_end] = y[idx_start:idx_end] + window*x["frames_filtered"]
    
df_frames[["frames_filtered"]].apply(overlap_add, axis=1)
show_signal(y, samplerate)

In [ ]:
wavfile.write("beat_processed.wav", samplerate, (y*2**16).astype(np.int16))